In [55]:
import pandas as pd

url = "https://raw.githubusercontent.com/martj42/international_results/master/results.csv"
df = pd.read_csv(url)
afcon=df

In [56]:
qualified = {
    'Morocco', 'Mali', 'Zambia', 'Comoros',
    'Egypt', 'South Africa', 'Angola', 'Zimbabwe',
    'Nigeria', 'Tunisia', 'Uganda', 'Tanzania',
    'Senegal', 'DR Congo', 'Benin', 'Botswana',
    'Algeria', 'Burkina Faso', 'Equatorial Guinea', 'Sudan',
    'Côte d’Ivoire', 'Cameroon', 'Gabon', 'Mozambique'
}



In [57]:
NAME_MAP = {
    "Ivory Coast": "Côte d’Ivoire",
    "Cote d'Ivoire": "Côte d’Ivoire",
    "Côte d'Ivoire": "Côte d’Ivoire",
    "Congo DR": "DR Congo",
    "DR Congo": "DR Congo",
    "South Africa": "South Africa",
    "Equatorial Guinea": "Equatorial Guinea",
}

In [58]:
def norm_team(name: str) -> str:
    name = str(name).strip()
    return NAME_MAP.get(name, name)

In [59]:

afcon["home_team_n"] = afcon["home_team"].map(norm_team)
afcon["away_team_n"] = afcon["away_team"].map(norm_team)
qualified_n = {norm_team(t) for t in qualified}

In [60]:
afcon_q = afcon[
    afcon["home_team_n"].isin(qualified_n) &
    afcon["away_team_n"].isin(qualified_n)
].copy()

In [61]:
afcon_q["date"] = pd.to_datetime(afcon_q["date"], errors="coerce")  # adapte dayfirst si besoin

# (optionnel) supprimer les dates invalides
afcon_q = afcon_q.dropna(subset=["date"])

In [63]:
afcon_q = afcon_q.loc[(afcon_q["date"].dt.year >= 2024) & (afcon_q["date"].dt.year <= 2025)].copy()

In [ ]:
import pandas as pd
import numpy as np

df = afcon_q.copy()

# Sécurité: scores en numérique
afcon_q["home_score"] = pd.to_numeric(afcon_q["home_score"], errors="coerce")
afcon_q["away_score"] = pd.to_numeric(afcon_q["away_score"], errors="coerce")

# Garder seulement les matchs avec scores valides
afcon_q = afcon_q.dropna(subset=["home_team", "away_team", "home_score", "away_score"])
# --- Stats côté domicile ---
home = pd.DataFrame({
    "team": df["home_team"],
    "goals_for": df["home_score"],
    "goals_against": df["away_score"],
    "wins": (df["home_score"] > df["away_score"]).astype(int),
    "draws": (df["home_score"] == df["away_score"]).astype(int),
    "losses": (df["home_score"] < df["away_score"]).astype(int),
})

# --- Stats côté extérieur ---
away = pd.DataFrame({
    "team": df["away_team"],
    "goals_for": df["away_score"],
    "goals_against": df["home_score"],
    "wins": (df["away_score"] > df["home_score"]).astype(int),
    "draws": (df["away_score"] == df["home_score"]).astype(int),
    "losses": (df["away_score"] < df["home_score"]).astype(int),
})

# --- Agrégation finale par équipe ---
team_stats = (
    pd.concat([home, away], ignore_index=True)
      .groupby("team", as_index=False)
      .sum(numeric_only=True)
)


team_stats["matches"] = team_stats["wins"] + team_stats["draws"] + team_stats["losses"]
team_stats["goal_diff"] = team_stats["goals_for"] - team_stats["goals_against"]

# Trier (ex: victoires puis diff de buts)
team_stats = team_stats.sort_values(["wins", "goal_diff", "goals_for"], ascending=False)

team_stats


,team,goals_for,goals_against,wins,draws,losses,matches,goal_diff
13,Morocco,27,6,12,1,1,14,21
0,Algeria,26,12,7,4,0,11,14
15,Nigeria,23,11,6,6,2,14,12
16,Senegal,15,6,5,4,0,9,9
17,South Africa,20,12,5,9,1,15,8
7,DR Congo,14,11,5,8,3,16,3
11,Ivory Coast,11,10,5,3,2,10,1
12,Mali,13,6,4,2,2,8,7
4,Burkina Faso,14,12,4,3,4,11,2
8,Egypt,11,4,3,3,0,6,7


In [65]:
df.to_csv("statistique_equipe.csv", index=False)
